# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# checking the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821




## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data will appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance in the local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Setting Keyspace

In [9]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Since in Apache Cassandra we must model our database tables with the queries we want to run in mind, we will create a table for each of the following queries. 

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [10]:
# Preview column order in file

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    for i in range(1):
        row = next(csvreader)
        for column in enumerate(row):
            print(column)

(0, 'artist')
(1, 'firstName')
(2, 'gender')
(3, 'itemInSession')
(4, 'lastName')
(5, 'length')
(6, 'level')
(7, 'location')
(8, 'sessionId')
(9, 'song')
(10, 'userId')


## QUERY #1
## Output the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [11]:
# Creating table for query #1

query = "CREATE TABLE IF NOT EXISTS music_library_query_1 "
query = query + "(sessionID int, ItemInSession int, artist text, song_title text, song_length float, PRIMARY KEY (sessionID, ItemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
# Loading data into music_library_query_1

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    
    for line in csvreader:
        query = "INSERT INTO music_library_query_1 (sessionID, ItemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [13]:
# Verifying ALL data was entered into the table
query = "SELECT * FROM music_library_query_1 LIMIT 5"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.sessionid, row.iteminsession, row.artist, row.song_title, row.song_length)

23 0 Regina Spektor The Calculation (Album Version) 191.08526611328125
23 1 Octopus Project All Of The Champs That Ever Lived 250.95791625976562
23 2 Tegan And Sara So Jealous 180.06158447265625
23 3 Dragonette Okay Dolores 153.39056396484375
23 4 Lil Wayne / Eminem Drop The World 229.58975219726562


#### Output the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [14]:
# Query #1

query = "SELECT artist, song_title, song_length FROM music_library_query_1 WHERE sessionid = 338 AND iteminsession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### QUERY #2

### Output the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [15]:
# Creating table
query = "CREATE TABLE IF NOT EXISTS music_library_query_2 "
query = query + "(userId int, sessionId int, ItemInSession int, artist text, song_title text, firstName text, lastName text, PRIMARY KEY ((userId, sessionID), ItemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [16]:
# Loading data into music_library2
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    
    for line in csvreader:
        query = "INSERT INTO music_library_query_2 (userId, sessionId, ItemInSession, artist, song_title, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [17]:
# Verifying that ALL data was entered into the table
query = "SELECT * FROM music_library_query_2 LIMIT 5"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.userid, row.sessionid, row.iteminsession, row.artist, row.song_title, row.firstname, row.lastname)

58 768 0 System of a Down Sad Statue Emily Benson
58 768 1 Ghostland Observatory Stranger Lover Emily Benson
58 768 2 Evergreen Terrace Zero Emily Benson
85 776 2 Deftones Head Up (LP Version) Kinsley Young
85 776 3 The Notorious B.I.G. Playa Hater (Amended Version) Kinsley Young


### Output the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [18]:
# Query #2

query = "SELECT artist, song_title, firstname, lastname FROM music_library_query_2 WHERE userid = 10 AND sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query #3

### Output every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [19]:
# Create table
query = "CREATE TABLE IF NOT EXISTS music_library_query_3 "
query = query + "(song_title text, firstName text, lastName text, PRIMARY KEY (song_title, firstName, lastName))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [20]:
# Loading data into music_library3
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    
    for line in csvreader:
        query = "INSERT INTO music_library_query_3 (song_title, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4]))

In [21]:
# Verifying the data was entered into the table3
query = "SELECT * FROM music_library_query_3 LIMIT 5"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song_title, row.firstname, row.lastname)

Wonder What's Next Chloe Cuevas
In The Dragon's Den Chloe Cuevas
Too Tough (1994 Digital Remaster) Aleena Kirby
Rio De Janeiro Blue (Album Version) Chloe Cuevas
My Place Jacob Klein


### Output every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [24]:
# Verifying QUERY #3

query = "SELECT firstname, lastname FROM music_library_query_3 WHERE song_title = 'All Hands Against His Own' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [25]:
# Dropping the tables

query = "drop table music_library_query_1"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table music_library_query_2"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table music_library_query_3"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [26]:
session.shutdown()
cluster.shutdown()